# Cognee x Memgraph integration

This notebook demonstrates how to integrate
[Cognee](https://github.com/cognee-ai/cognee) with
[Memgraph](https://memgraph.com), a graph database platform, to automatically
convert unstructured text into a semantically searchable knowledge graph using
Large Language Models (LLMs). 

Cognee is an AI-powered toolkit for cognitive search and graph-based knowledge
representation. It uses LLMs to break down natural language into structured
concepts and relationships, storing them as graphs in Memgraph for further
querying and visualization.

In this example, we'll take you through:
- Setting up the environment with Docker and Poetry
- Connecting Cognee to Memgraph
- Adding unstructured text content
- Generating a knowledge graph using LLMs
- Performing semantic search over the graph

By the end, you’ll be able to:
- Create a local graph database instance with Memgraph
- Automatically build and store knowledge graphs from plain text
- Query your knowledge graph using natural language


## Prerequisites

To follow along, you'll need:

1. **Docker**: Ensure [Docker](https://www.docker.com/) is installed and running
   in the background. 

2. **Memgraph**: The easiest way to run Memgraph is using the following
   commands:

For Linux/macOS: `curl https://install.memgraph.com | sh`

For Windows: `iwr https://windows.memgraph.com | iex`

This will launch Memgraph at `localhost:3000`.

3. Install neccessary dependencies, open your terminal and run:
```bash
poetry init
poetry add cognee
pip install neo4j
```

## Environment setup

We'll load the environment variables used to configure the LLM and graph
database providers. These will be pulled from a `.env` file (which you must
create securely — don’t share API keys!). In this example, we're using OpenAI.

Create a file named `.env` in your project root with the following content:

```env
LLM_API_KEY=sk-...
LLM_MODEL=openai/gpt-4o-mini
LLM_PROVIDER=openai
EMBEDDING_PROVIDER=openai
EMBEDDING_MODEL=openai/text-embedding-3-large
GRAPH_DATABASE_PROVIDER=memgraph
GRAPH_DATABASE_URL=bolt://localhost:7687
GRAPH_DATABASE_USERNAME=" "
GRAPH_DATABASE_PASSWORD=" "
```

## Create your script

Let's first load our environment in notebook:

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

We’ll add a small paragraph of text to Cognee. This will be processed to extract semantic entities and relationships.

In [ ]:
import cognee
import asyncio

async def run():
    text = "Natural language processing (NLP) is a subfield of computer science focused on the interaction between computers and human language."

    # Step 1: Add text to Cognee
    await cognee.add(text)

    # Step 2: Build the knowledge graph using LLMs
    await cognee.cognify()

    # Step 3: Perform a semantic search
    results = await cognee.search(query_text="What is NLP?")
    
    print("Search Results:\n")
    for result in results:
        print(result)

await run()


### What happens under the hood?

- `cognee.add()` stores the input text internally.
- `cognee.cognify()` uses the configured LLM (e.g., GPT-4o-mini) to extract
  entities, facts, and relations from the text and converts them into a graph
  structure.
- `cognee.search()` enables semantic querying, meaning the engine interprets the
  intent of your question and maps it to relevant parts of the graph.

## Visualize your data in Memgraph

Now that the graph is created, we can explore it in the UI by visiting
`http://localhost:3000/`.

The easiest way to visualize the entire graph is by running a Cypher command
similar to this:

`MATCH p=()-[]-() RETURN p;`

This command matches all of the possible paths in the graph and returns entire
graph.

To visualize the schema of the graph, visit the Graph schema tab and generate
the new schema based on the newly created graph.

To delete an entire graph, use:

`MATCH (n) DETACH DELETE n;`


## Conclusion

In this notebook, we successfully demonstrated how to convert unstructured
natural language text into a structured, queryable knowledge graph using Cognee
and Memgraph. By leveraging the power of large language models, we extracted
semantic relationships from raw text and stored them in a graph database for
exploration and analysis.

This integration offers a powerful foundation for building intelligent systems
that can understand, reason over, and retrieve knowledge in a human-like way.
Whether you're working with internal documents, research papers, or customer
support data, this workflow enables you to unlock insights from language using
cutting-edge AI and graph technologies.